In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Code LLaMA

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_codellama.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_codellama.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_codellama.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a> (A Python-3 CPU notebook is recommended)
  </td>
</table>

## Overview

This notebook demonstrates deploying pretrained Code LLaMA models in Vertex AI.

### Objective

- Deploy pre-trained Code LLaMA models with [vLLM](https://github.com/vllm-project/vllm) with best serving throughput

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

### Colab only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    ! pip3 install ipython pandas[output_formatting] google-cloud-language==2.10.0
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API, Compute Engine API and Cloud Natural Language API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,language.googleapis.com).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
GCS_BUCKET = ""  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud services enable language.googleapis.com

import os

STAGING_BUCKET = os.path.join(GCS_BUCKET, "staging")
MODEL_BUCKET = os.path.join(GCS_BUCKET, "code-llama")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

from google.colab import auth

auth.authenticate_user(project_id=PROJECT_ID)

### Initialize Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker images.
VLLM_DOCKER_URI = (
    "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve"
)

### Define common functions

In [ ]:
import os
from datetime import datetime

from google.cloud import aiplatform


def get_job_name_with_datetime(prefix: str):
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model_vllm(
    model_name,
    model_id,
    service_account,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
):
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.95",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

## Access pretrained Code LLaMA models
The original models from Meta are converted into the HuggingFace format for serving in Vertex AI.
After clicking the agreement of Code LLaMA model card in Vertex AI Model Garden, a Cloud Storage bucket will be shared to access pretrained Code LLaMA models.

In [ ]:
VERTEX_AI_MODEL_GARDEN_CODE_LLAMA = (
    ""  # This path will be shared once click the agreement in Vertex AI Model Garden.
)
assert (
    VERTEX_AI_MODEL_GARDEN_CODE_LLAMA
), "Please click the agreement of Code LLaMA in Vertex AI Model Garden, and get the GCS path of Code LLaMA model artifacts."
print(
    "Copy Code LLaMA model artifacts from",
    VERTEX_AI_MODEL_GARDEN_CODE_LLAMA,
    "to ",
    MODEL_BUCKET,
)
! gsutil -m cp -R $VERTEX_AI_MODEL_GARDEN_CODE_LLAMA/* $MODEL_BUCKET

In [ ]:
model_name = "CodeLlama-7b-Instruct-hf"  # @param ["CodeLlama-7b-hf", "CodeLlama-7b-Python-hf", "CodeLlama-7b-Instruct-hf", "CodeLlama-13b-hf", "CodeLlama-13b-Python-hf", "CodeLlama-13b-Instruct-hf", "CodeLlama-34b-hf", "CodeLlama-34b-Python-hf", "CodeLlama-34b-Instruct-hf"]
model_id = os.path.join(MODEL_BUCKET, model_name)
print(model_id)

## Deploy Pretrained Code LLaMA with vLLM

This section deploys prebuilt OpenLLaMA models with [vLLM](https://github.com/vllm-project/vllm) on the Endpoint. The model deployment step will take ~15 minutes to complete.

The vLLM project is an highly optimized LLM serving framework which can increase serving throughput a lot. The higher QPS you have, the more benefits you get using vLLM.

In [ ]:
# Since all models are stored in bfloat16 precision, A100 is needed for serving.

# Sets A100 (40G) to deploy 7B.
machine_type = "a2-highgpu-1g"
accelerator_type = "NVIDIA_TESLA_A100"
accelerator_count = 1

# Sets A100 (40G) to deploy 13B.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Sets 2 A100 (40G) to deploy 34B models.
# machine_type = "a2-highgpu-2g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 2

model_vllm, endpoint_vllm = deploy_model_vllm(
    model_name=get_job_name_with_datetime(prefix="code-llama-serve-vllm"),
    model_id=model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)

NOTE: The prebuilt model weights will be downloaded on the fly from the $MODEL_BUCKET after the deployment succeeds. Thus additional 5 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

Once deployment succeeds, you can send requests to the endpoint with text prompts.

In [ ]:
instance = {
    "prompt": "import argparse",
    "n": 1,
    "max_tokens": 200,
}
response = endpoint_vllm.predict(instances=[instance])
print(response.predictions[0])

Additionaly, you can send requests to the endpoint and get streaming response from it.

In [ ]:
import json

import requests


def get_streaming_response(response: requests.Response):
    for chunk in response.iter_lines(
        chunk_size=8192, decode_unicode=False, delimiter=b"\0"
    ):
        if chunk:
            data = json.loads(chunk.decode("utf-8"))
            yield data


instance = {
    "prompt": "import argparse",
    "n": 1,
    "max_tokens": 200,
}
response = endpoint_vllm.raw_predict(
    body=json.dumps({"instances": [instance]}),
    headers={"Content-Type": "application/json"},
)

print("Prompt:\n", prompt)
text_len = 0
print("Streaming:")
for output in get_streaming_response(response):
    text = output["predictions"][0]
    print(text[text_len:])
    text_len = len(text)
print("Output:\n", text)

## Clean up resources

In [ ]:
# Undeploy model and delete endpoint.
endpoint_vllm.delete(force=True)

# Delete models.
model_vllm.delete()

# Uncomment below to delete all artifacts
# !gsutil -m rm -r $STAGING_BUCKET $MODEL_BUCKET